In [2]:
from agents import Agent, function_tool, Runner

In [3]:
import requests

In [4]:
runner=Runner()

In [5]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner
chat_interface = IPythonChatInterface()

In [6]:
# def fetch_webpage_tool(url):
#     jina_reader_base_url = "https://r.jina.ai/"
#     jina_reader_url = jina_reader_base_url + url
#     response = requests.get(jina_reader_url)
#     return response.content.decode('utf-8')

### search agent

In [7]:
from minsearch import AppendableIndex
import docs

In [8]:
index = AppendableIndex(text_fields=['content'])

In [9]:
from typing import Any, Optional, Dict, List
from requests.exceptions import RequestException

class SearchTools:
    def __init__(self, index: Any) -> None:
        """
        Initialize the SearchTools with an index object.

        Args:
            index (Any): An index object or a list to store chunks.
                         Must implement `append(chunk)` if it's a list,
                         or `search(query, num_results)` if a search index.
        """
        self.index = index

    def index_page(self, url: str, content: Optional[str] = None) -> bool:
        """
        Fetch and index the content of a web page.

        Args:
            url (str): The URL of the web page.
            content (Optional[str]): If provided, use this content instead of fetching.

        Returns:
            bool: True if indexing succeeds, False otherwise.
        """
        try:
            if not content:
                content = fetch_webpage_tool(url)
                if not content:
                    print(f"No content found at the URL: {url}")
                    return False

            # Chunk the content using a sliding window approach
            chunks: List[Dict[str, Any]] = docs.sliding_window(content, 3000, 1000)
            for chunk in chunks:
                chunk["url"] = url

                # Append to index; works if index is a list or supports append
                if hasattr(self.index, "append"):
                    self.index.append(chunk)
                else:
                    raise AttributeError("Index object does not support 'append' method.")

            return True

        except (RequestException, ValueError) as e:
            print(f"Error indexing web page {url}: {e}")
            return False
        except Exception as e:
            print(f"Unexpected error during indexing: {e}")
            return False

    def search(self, query: str) -> Dict[str, Any]:
        """
        Search for relevant results in the indexed content.

        Args:
            query (str): The user's search query.

        Returns:
            Dict[str, Any]: A dictionary containing the top search results, or empty if search fails.
        """
        try:
            if hasattr(self.index, "search"):
                results = self.index.search(query, num_results=5)
                return results
            else:
                raise AttributeError(
                    "Index object must implement a 'search(query, num_results=...)' method."
                )
        except Exception as e:
            print(f"Error performing search: {e}")
            return {}


In [10]:
search_tools=SearchTools(index)

In [11]:
from toyaikit.tools import wrap_instance_methods
search_method_tools= wrap_instance_methods(function_tool,search_tools)

In [12]:
search_instructions = """
Your task is to search through indexed documents.
Before performing a search:
- Check if the url has been indexed.if not then call  index_page
- When asked to index multiple pages, fetch and index them **one by one**.
- Use the **Fetch Web Page** tool to retrieve each page’s content.
- Pass that content to the **Index Page** function to store it for later search or summarization.
- Confirm successful indexing for each page.
only do a search if the user provides a question not URL .Do Not summarize it

"""

search_agent = Agent(
    name='search_agent',
    instructions=search_instructions,
    handoff_description="Whenever the user needs to search for things from url ",
    tools=search_method_tools,
    model='gpt-4o-mini'
)

In [13]:
runner=OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=search_agent
)

#### summary agent

In [14]:
import requests
from requests.exceptions import RequestException

def fetch_webpage_tool(url: str) -> str:
    """
    Fetch the HTML content of a webpage using the Jina reader base URL.

    Args:
        url (str): The path or endpoint to append to the Jina reader base URL.

    Returns:
        str: The decoded HTML content of the fetched webpage.

    Raises:
        ValueError: If the URL is empty or None.
        RuntimeError: If there is a network issue or the request fails.
    """
    if not url:
        raise ValueError("The URL parameter must not be empty or None.")

    jina_reader_base_url = "https://r.jina.ai/"
    jina_reader_url = jina_reader_base_url + url

    try:
        response = requests.get(jina_reader_url)
        response.raise_for_status()  # Raise HTTPError for bad responses
    except RequestException as e:
        raise RuntimeError(f"Failed to fetch the webpage: {e}") from e

    try:
        content = response.content.decode('utf-8')
    except UnicodeDecodeError as e:
        raise RuntimeError(f"Failed to decode webpage content: {e}") from e

    return content


In [15]:
class SummaryTools:
    def __init__(self,search_tools):
        self.search_tools=search_tools
        self.data = []
    def fetch_content(self, url: str) -> str:
        """
        Fetch the HTML content of a webpage using the Jina reader base URL.

        Args:
            url (str): The path or endpoint to append to the Jina reader base URL.

        Returns:
            str: The content of the fetched webpage.
        """
        content = fetch_webpage_tool(url)
        self.search_tools.index_page(url,content)
        
        return content
        
            
    def save_summary_tool(self, summary: str) -> None:
         
         """Append the summary to memory and a text file."""
         self.data.append(summary)
         with open("summaries.txt", "a", encoding="utf-8") as f:
             f.write(summary + "\n")
         print(" Summary saved to summaries.txt")
       

In [16]:
summary_tools=SummaryTools(search_tools)

In [17]:
summary_method_tools = wrap_instance_methods(function_tool,summary_tools)

In [18]:
# content = fetch_webpage_tool("https://en.wikipedia.org/wiki/Capybara")
# print(content)

In [19]:
summary_instructions = """
You are an intelligent assistant with access to two tools: 

Your task is to summarizes  about webpages. Follow these steps:  

1. When asked "What is this page about?" first use the **fetch content** tool with the provided URL to get the page content and index it .  
2. Read the content and generate a concise summary of what the page is about.  
3. Use the **Save Summary** tool to save the summary in a .txt file.  
4. Return the summary as your answer to the user.  

Always ensure the summary is clear, concise, and accurate. Do not skip any of the steps.
"""

In [20]:
summary_agent = Agent(
    name = 'summary_agent',
    instructions = summary_instructions,
    handoff_description = "whenever the user asks what is the page about",
    tools=summary_method_tools,
    model='gpt-4o-mini'
)

In [21]:
from agents import handoff

In [22]:
triage_instructions = """
You are the orchestrator between two specialized agents:

1. summarizing_agent — summarizes web pages .
2. search_agent — searches within previously indexed documents to answer detailed or follow-up questions.

Routing rules:

- If the user sends a url and asks "What is this page about?  → hand off to summary_agent.
- If the user asks a follow-up question after a summary (e.g. "how did they do X?", "what does she say about Y?"),
  or refers to something mentioned in a previously summarized or indexed resource → hand off to search_agent.
- if the user gives many urls to index it -> hand off to search agent
- If the user asks a direct content question about a topic inside a resource → search_agent.
- Prefer delegating the answer to agents when possible. 

Examples:
User: ""What is this page about? https://en.wikipedia.org/wiki/Capybara"
→ summary_agent

User: "What does the threat?"
→ search_agent

User: "How exactly did they fight malaria?" (after a summary)
→ search_agent
""".strip()
triage_agent = Agent(
    name='triage_agent',
    instructions=triage_instructions,
    handoffs=[
        handoff(summary_agent, on_handoff=lambda ctx: print('handoff to summary agent')),
        handoff(search_agent, on_handoff=lambda ctx: print('handoff to search agent')),
    ],
    model='gpt-4o-mini'
)

In [23]:
runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=triage_agent
)

await runner.run();

You: What is this page about? https://en.wikipedia.org/wiki/Capybara


handoff to summary agent
 Summary saved to summaries.txt
handoff: transfer_to_summary_agent
handoff: summary_agent -> triage_agent successful


You: index these pages too:  Lesser capybara — https://en.wikipedia.org/wiki/Lesser_capybara  Hydrochoerus (genus) — https://en.wikipedia.org/wiki/Hydrochoerus  Neochoerus (extinct genus related to capybaras) — https://en.wikipedia.org/wiki/Neochoerus  Caviodon (extinct genus of rodents related to capybaras) — https://en.wikipedia.org/wiki/Caviodon  Neochoerus aesopi (extinct species close to capybaras) — https://en.wikipedia.org/wiki/Neochoerus_aesopi


handoff to search agent
handoff: transfer_to_search_agent
handoff: search_agent -> triage_agent successful


You: What are threats to capybara populations?


handoff to search agent
handoff: transfer_to_search_agent
handoff: search_agent -> triage_agent successful


You: stop


Chat ended.
